# LangChain Expression Language (LCEL)

A Linguagem de Expressão LangChain, ou LCEL, é uma forma declarativa de **compor cadeias de maneira fácil**. A LCEL foi projetada desde o primeiro dia para suportar a colocação de protótipos em produção, sem a necessidade de alterações no código, desde a cadeia mais simples “prompt + LLM” até as cadeias mais complexas (já vimos pessoas executando com sucesso cadeias LCEL com centenas de etapas em produção). Para destacar alguns dos motivos pelos quais você pode querer usar a LCEL:
- **Suporte a streaming de primeira classe**: menor tempo possível para saída do primeiro token produzidio;
- **Suporte assíncrono**: Qualquer cadeia construída com a LCEL pode ser chamada tanto com a API síncrona;
- **Execução paralela otimizada**: Sempre que suas cadeias LCEL tiverem etapas que podem ser executadas em paralelo, automaticamente é feito isso;
- **Retentativas e fallbacks**: É maneira de tornar suas cadeias mais confiáveis em grande escala, na qual ações alternativas podem ser tomadas no caso de um erro em uma cadeia
- **Acessar resultados intermediários**: auxiliando na depuração de uma cadeia;

## Um exemplo simples de LCEL

O exemplo mais simples que podemos mostrar seria na construção de *prompt + model*
Anotação Evandro: este trecho abaixo serve para carregar o arquivo .env no caso as keys, muito importante para não deixar as keys explicitas no codigo


In [1]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [3]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template('Crie uma frase sobre o seguinte assunto: {assunto}')

In [4]:
chain = prompt | model
chain.invoke({'assunto': 'futebol'})

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

### Adicinando mais elementos a chain

In [5]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | model | StrOutputParser()
chain.invoke({'assunto': 'futebol'})

'O futebol une pessoas de diferentes culturas e nacionalidades em um só amor pelo esporte.'

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template('Crie uma frase sobre o seguinte: {assunto}')

chain = prompt | ChatOpenAI() | StrOutputParser()
chain.invoke({'assunto': 'gatinhos'})

'Os gatinhos são seres adoráveis e cheios de charme, capazes de conquistar qualquer coração com seu ronronar suave e olhar meigo.'

### ATENÇÃO! A ordem importa

In [7]:
chain =  ChatOpenAI() | prompt | StrOutputParser()
chain.invoke({'assunto': 'gatinhos'})

ValueError: Invalid input type <class 'dict'>. Must be a PromptValue, str, or list of BaseMessages.

### Como eu sei a ordem?

Para atingir o mesmo objetivo sem a criação da chain, os passos que eu deveria seguir seriam:
- Formatar o prompt template
- Enviar o prompt formatado para o modelo
- Fazer o parseamento do saída do modelo
Essa mesma ordem deve ser seguida para não termos erros

In [8]:
input = {'assunto': 'gatinhos'}

In [9]:
prompt_formatado = prompt.invoke(input)
prompt_formatado

ChatPromptValue(messages=[HumanMessage(content='Crie uma frase sobre o seguinte: gatinhos', additional_kwargs={}, response_metadata={})])

In [10]:
resposta = model.invoke(prompt_formatado)
resposta

AIMessage(content='Os gatinhos são como pequenos raios de sol que iluminam nossos dias com sua fofura e travessuras.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 19, 'total_tokens': 50, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3d9fe243-aa00-4b1f-85b6-1e50d30ed02d-0', usage_metadata={'input_tokens': 19, 'output_tokens': 31, 'total_tokens': 50, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [11]:
StrOutputParser().invoke(resposta)

'Os gatinhos são como pequenos raios de sol que iluminam nossos dias com sua fofura e travessuras.'

### Para não errar a ordem

É importante entendermos que cada componente recebe um tipo de entrada e gera um tipo de saída, e estes tipos precisam casar:

| Component       | Tipo de Entrada                                    | Tipo de Saída        |
|-----------------|-----------------------------------------------------|----------------------|
| Prompt          | Dicionário                                         | PromptValue      |
| ChatModel       | String única, lista de mensagens de chat ou PromptValue | Mensagem de Chat     |
| LLM             | String única, lista de mensagens de chat ou PromptValue | String               |
| OutputParser    | A saída de um LLM ou ChatModel                     | Depende do parser    |
| Retriever       | String única                                       | Lista de Documentos  |
| Tool            | String única ou dicionário, dependendo da ferramenta| Depende da ferramenta|

Podemos verificar isso pelos output e input schemas:

In [12]:
prompt.input_schema.model_json_schema()

{'properties': {'assunto': {'title': 'Assunto', 'type': 'string'}},
 'required': ['assunto'],
 'title': 'PromptInput',
 'type': 'object'}

In [13]:
prompt.output_schema.model_json_schema()

{'$defs': {'AIMessage': {'additionalProperties': True,
   'description': 'Message from an AI.\n\nAIMessage is returned from a chat model as a response to a prompt.\n\nThis message represents the output of the model and consists of both\nthe raw output as returned by the model together standardized fields\n(e.g., tool calls, usage metadata) added by the LangChain framework.',
   'properties': {'content': {'anyOf': [{'type': 'string'},
      {'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]},
       'type': 'array'}],
     'title': 'Content'},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'response_metadata': {'title': 'Response Metadata', 'type': 'object'},
    'type': {'const': 'ai',
     'default': 'ai',
     'title': 'Type',
     'type': 'string'},
    'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'title': 'Name'},
    'id': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     '

### Como eu teria feito com chains clássicas?

In [14]:
from langchain.chains.llm import LLMChain
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(model="gpt-3.5-turbo")
prompt = ChatPromptTemplate.from_template("Crie uma frase sobre o assunto: {assunto}")
output_parser = StrOutputParser()

In [15]:
chain = LLMChain(
    llm=model,
    prompt=prompt,
    output_parser=output_parser
)
chain.invoke({'assunto': 'gatinhos'})

/tmp/ipykernel_228436/1936627178.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


{'assunto': 'gatinhos',
 'text': 'Os gatinhos são seres adoráveis e cheios de personalidade, capazes de conquistar até os corações mais duros.'}